In [24]:
# run all

from bs4 import BeautifulSoup
import urllib.request
import re
import pandas as pd
from datetime import datetime
import time
from sklearn.externals import joblib
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

# scrape current data
url="http://www.leagueofgraphs.com/champions/stats/diamond"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page.read(), "lxml")

for tr in soup.find_all('tr')[1:145]:
    tds = tr.find_all('td')
    
    if len(tds) >1:
        name = tds[1].text.split()
    
        champ_name = name[0].lower().replace("'", '').replace(",", '').replace(".", '')
        if champ_name == 'lee':
            champ_name = 'leesin'
        elif champ_name == 'miss':
            champ_name = 'missfortune'
        elif champ_name == 'twisted':
            champ_name = 'twistedfate'
        elif champ_name == 'master':
            champ_name = 'masteryi'
        elif champ_name == 'xin':
            champ_name = 'xinzhao'
        elif champ_name == 'dr':
            champ_name = 'drmundo'
        elif champ_name == 'tahm':
            champ_name = 'tahmkench'
        elif champ_name == 'aurelion':
            champ_name = 'aurelionsol'              
        elif champ_name == 'jarvan':
            champ_name = 'jarvaniv' 
            
        pop = float(tds[2].text.split()[0][0:-1])
        winrate = tds[3].text.split()[0][0:-1]
        banrate = tds[4].text.split()[0][0:-1]
        
#         print(champ_name, str(pop), str(winrate), str(banrate))
        
        with open('lolcurrent_raw_20170207.csv', 'a') as f:
            s = champ_name + ',' + str(pop) + ',' +  str(winrate) + ',' + \
                str(banrate) + '\n'
            f.write(s)
            

# add time since released feature
df = pd.read_csv('lolcurrent_raw_20170207.csv', names = ['name', 'popularity', 'winrate',
                                           'banrate'])
df_release = pd.read_csv('lolrelease.csv', names=['name', 'release'])

df = df.merge(df_release, how='outer', on='name')
df.dropna(inplace=True, axis=0)
date = datetime.now()
date = time.mktime(date.timetuple())

df['tsr'] = df.apply(lambda x: date-x.release, axis=1)

# add time since last nerf buff
df_all = pd.read_csv('loldata_cleaned_20170207.csv')
lastall = df_all[df_all['date']==df_all['date'].max()][['name', 'ts_buff', 'ts_nerf', 'popularity', 'winrate','banrate']]

lastall['ts_buff'] = lastall['ts_buff'].apply(lambda x: x+1 if x <25 else 25)
lastall['ts_nerf'] = lastall['ts_nerf'].apply(lambda x: x+1 if x <25 else 25)
lastall.sort_values('name', inplace=True)
lastall.reset_index(inplace=True, drop=True)

df = df.merge(lastall[['name', 'ts_buff', 'ts_nerf']], on='name')
df.sort_values('name', inplace=True)
df.reset_index(inplace=True, drop=True)

# add velocity
df['vpop'] = df['popularity']-lastall['popularity']
df['vwr'] = df['winrate']-lastall['winrate']
df['vbr'] = df['banrate']-lastall['banrate']

df.to_csv('lolcurrent.csv', index=False)

In [35]:
# run model
df = pd.read_csv('lolcurrent.csv')
xgbc = joblib.load('xgbc.dat') 
p_xgb_o = xgbc.predict(df[['popularity', 'winrate', 'banrate',
                            'tsr', 'ts_buff', 'ts_nerf', 'vpop', 'vbr', 'vwr']].as_matrix())
prob_xgb_o = xgbc.predict_proba(df[['popularity', 'winrate', 'banrate', 'tsr', 'ts_buff', 'ts_nerf', 'vpop','vbr', 'vwr']].as_matrix())
dfp = pd.DataFrame(p_xgb_o, columns=['changes'])

dfn = pd.DataFrame(df.iloc[dfp[dfp['changes'] == 'nerf'].index]['name'])
# nerfs_html = dfn.transpose().to_html(index=False)

dfb = pd.DataFrame(df.iloc[dfp[dfp['changes'] == 'buff'].index]['name'])
# buffs_html = dfb.transpose().to_html(index=False)

# Format probabilities for predicted nerfs
df_nerf_probs = pd.read_csv('lolcurrent_nerf_probs.csv')
champs_nerf = list(dfn['name'].values)
champs_nerf.append('dates')
# df_nerf_probs = df_nerf_probs[champs_nerf]
df_nerf_probs['dates'] = pd.to_datetime(df_nerf_probs['dates'],unit='s').dt.date
df_nerf_probs.to_csv('graph_nerfs_all.csv', index=False, na_rep='NaN')

# Format probabilities for predicted buffs
df_buff_probs = pd.read_csv('lolcurrent_buff_probs.csv')
champs_buff = list(dfb['name'].values)
champs_buff.append('dates')
# df_buff_probs = df_buff_probs[champs_buff]
df_buff_probs['dates'] = pd.to_datetime(df_buff_probs['dates'],unit='s').dt.date
df_buff_probs.to_csv('graph_buffs_all.csv', index=False, na_rep='NaN')



In [51]:
print(prob_xgb_o[dfp[dfp['changes'] == 'nerf'].index])
df.iloc[dfp[dfp['changes'] == 'nerf'].index]['name']

[[ 0.11882934  0.78195482  0.09921588]
 [ 0.06263418  0.57631546  0.36105034]
 [ 0.11110085  0.63041234  0.25848678]
 [ 0.23816136  0.58930284  0.17253581]
 [ 0.25502101  0.43929845  0.30568057]
 [ 0.08895427  0.80732983  0.1037159 ]
 [ 0.13355261  0.57523286  0.29121456]
 [ 0.119775    0.79184473  0.08838028]
 [ 0.04123826  0.49433669  0.46442509]
 [ 0.07850027  0.47502774  0.44647196]]


15      camille
40        ivern
44        jayce
54       khazix
55      kindred
58      leblanc
66     malzahar
86       rengar
91        shaco
104       talon
Name: name, dtype: object

In [1]:
from bs4 import BeautifulSoup
import urllib.request
import re

url="http://www.leagueofgraphs.com/champions/stats/diamond"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page.read(), "lxml")



In [2]:
for tr in soup.find_all('tr')[1:145]:
    tds = tr.find_all('td')
    
    if len(tds) >1:
        name = tds[1].text.split()
    
        champ_name = name[0].lower().replace("'", '').replace(",", '').replace(".", '')
        if champ_name == 'lee':
            champ_name = 'leesin'
        elif champ_name == 'miss':
            champ_name = 'missfortune'
        elif champ_name == 'twisted':
            champ_name = 'twistedfate'
        elif champ_name == 'master':
            champ_name = 'masteryi'
        elif champ_name == 'xin':
            champ_name = 'xinzhao'
        elif champ_name == 'dr':
            champ_name = 'drmundo'
        elif champ_name == 'tahm':
            champ_name = 'tahmkench'
        elif champ_name == 'aurelion':
            champ_name = 'aurelionsol'              
        elif champ_name == 'jarvan':
            champ_name = 'jarvaniv' 
            
        pop = float(tds[2].text.split()[0][0:-1])
        winrate = tds[3].text.split()[0][0:-1]
        banrate = tds[4].text.split()[0][0:-1]
        
#         print(champ_name, str(pop), str(winrate), str(banrate))
        
        with open('lolcurrent_raw_20170207.csv', 'a') as f:
            s = champ_name + ',' + str(pop) + ',' +  str(winrate) + ',' + \
                str(banrate) + '\n'
            f.write(s)

In [4]:
import pandas as pd
from datetime import datetime
import time

df = pd.read_csv('lolcurrent_raw_20170207.csv', names = ['name', 'popularity', 'winrate',
                                           'banrate'])
df_release = pd.read_csv('lolrelease.csv', names=['name', 'release'])

# df_release.head()

,name,release
0,aatrox,1.370045e+09
1,ahri,1.322698e+09
2,akali,1.272672e+09
3,alistar,1.233446e+09
4,amumu,1.243814e+09


In [5]:
df = df.merge(df_release, how='outer', on='name')

In [7]:
df.dropna(inplace=True, axis=0)

In [8]:
df.head()

,name,popularity,winrate,banrate,release
0,jhin,44.3,52.9,1.2,1.451606e+09
1,leesin,35.8,48.9,7.3,1.270080e+09
2,varus,32.3,49.3,1.3,1.335830e+09
3,thresh,28.8,49.3,0.3,1.356998e+09
4,ezreal,27.1,47.5,0.2,1.267402e+09


In [9]:

date = datetime.now()
date = time.mktime(date.timetuple())

df['tsr'] = df.apply(lambda x: date-x.release, axis=1)

In [11]:
df.head()

,name,popularity,winrate,banrate,release,tsr
0,jhin,44.3,52.9,1.2,1.451606e+09,34886819.0
1,leesin,35.8,48.9,7.3,1.270080e+09,216413219.0
2,varus,32.3,49.3,1.3,1.335830e+09,150662819.0
3,thresh,28.8,49.3,0.3,1.356998e+09,129494819.0
4,ezreal,27.1,47.5,0.2,1.267402e+09,219091619.0


In [ ]:
# df_release = pd.read_csv('lolcurrent.csv', names=['name', 'release'])

In [13]:
# df.to_csv('lolcurrent.csv', index=False)

In [12]:
df_all = pd.read_csv('loldata_cleaned_20170207.csv')
df_all.head()

,banrate,changes,date,haschange,name,popularity,release,ts_buff,ts_nerf,tsr,vbr,vpop,vwr,winrate
0,0.129678,nochange,1.395140e+09,False,aatrox,4.34,1.370045e+09,25.0,25.0,2.509560e+07,-0.307640,-0.04,-2.02,49.72
1,0.193766,nochange,1.395767e+09,False,aatrox,4.80,1.370045e+09,25.0,25.0,2.572200e+07,0.064088,0.46,1.13,50.85
2,0.279357,nochange,1.396411e+09,False,aatrox,5.06,1.370045e+09,25.0,25.0,2.636640e+07,0.085591,0.26,0.03,50.88
3,0.203943,nochange,1.397171e+09,False,aatrox,4.53,1.370045e+09,25.0,25.0,2.712601e+07,-0.075414,-0.53,-1.40,49.48
4,0.340798,nochange,1.397808e+09,False,aatrox,4.53,1.370045e+09,25.0,25.0,2.776320e+07,0.136856,0.00,1.71,51.19


In [13]:
lastall = df_all[df_all['date']==df_all['date'].max()][['name', 'ts_buff', 'ts_nerf', 'popularity', 'winrate','banrate']]
lastall.head(10)

,name,ts_buff,ts_nerf,popularity,winrate,banrate
135,aatrox,25.0,25.0,0.59,42.73,0.760703
271,yorick,25.0,25.0,1.23,49.49,0.063091
406,cassiopeia,25.0,25.0,6.96,48.43,1.319513
542,draven,25.0,16.0,8.53,52.68,0.665164
678,lulu,25.0,12.0,12.27,51.13,0.162235
813,sona,25.0,25.0,4.02,51.77,0.016224
948,alistar,25.0,25.0,4.25,50.00,0.167643
1084,kassadin,25.0,25.0,3.23,49.13,0.126183
1219,soraka,25.0,25.0,8.64,52.97,0.376746
1355,rammus,25.0,25.0,1.80,49.50,0.086525


In [14]:
lastall['ts_buff'] = lastall['ts_buff'].apply(lambda x: x+1 if x <25 else 25)
lastall['ts_nerf'] = lastall['ts_nerf'].apply(lambda x: x+1 if x <25 else 25)
lastall.sort_values('name', inplace=True)
lastall.reset_index(inplace=True, drop=True)
lastall.head(10)

,name,ts_buff,ts_nerf,popularity,winrate,banrate
0,aatrox,25.0,25.0,0.59,42.73,0.760703
1,ahri,25.0,25.0,12.90,52.17,0.816584
2,akali,25.0,13.0,2.59,50.80,0.243353
3,alistar,25.0,25.0,4.25,50.00,0.167643
4,amumu,25.0,25.0,1.32,47.27,0.120775
5,anivia,23.0,25.0,3.24,54.39,0.153222
6,annie,25.0,25.0,3.93,51.10,0.317260
7,ashe,25.0,25.0,10.97,46.68,0.088328
8,aurelionsol,25.0,25.0,1.63,54.36,0.155025
9,azir,25.0,21.0,1.71,43.16,0.136999


In [15]:
df = df.merge(lastall[['name', 'ts_buff', 'ts_nerf']], on='name')
df.sort_values('name', inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,name,popularity,winrate,banrate,release,tsr,ts_buff,ts_nerf
0,aatrox,0.6,42.7,0.8,1.370045e+09,116448419.0,25.0,25.0
1,ahri,12.9,52.2,0.8,1.322698e+09,163795619.0,25.0,25.0
2,akali,2.6,50.8,0.2,1.272672e+09,213821219.0,25.0,13.0
3,alistar,4.3,50.0,0.2,1.233446e+09,253046819.0,25.0,25.0
4,amumu,1.3,47.3,0.1,1.243814e+09,242678819.0,25.0,25.0


In [16]:
df['vpop'] = df['popularity']-lastall['popularity']
df['vwr'] = df['winrate']-lastall['winrate']
df['vbr'] = df['banrate']-lastall['banrate']

In [18]:
df.head()

,name,popularity,winrate,banrate,release,tsr,ts_buff,ts_nerf,vpop,vwr,vbr
0,aatrox,0.6,42.7,0.8,1.370045e+09,116448419.0,25.0,25.0,0.01,-0.03,0.039297
1,ahri,12.9,52.2,0.8,1.322698e+09,163795619.0,25.0,25.0,0.00,0.03,-0.016584
2,akali,2.6,50.8,0.2,1.272672e+09,213821219.0,25.0,13.0,0.01,0.00,-0.043353
3,alistar,4.3,50.0,0.2,1.233446e+09,253046819.0,25.0,25.0,0.05,0.00,0.032357
4,amumu,1.3,47.3,0.1,1.243814e+09,242678819.0,25.0,25.0,-0.02,0.03,-0.020775


In [94]:
# df.to_csv('lolcurrent.csv', index=False)

In [12]:
from sklearn.externals import joblib
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import pandas as pd

df = pd.read_csv('lolcurrent.csv')
xgbc = joblib.load('xgbc.dat') 
p_xgb_o = xgbc.predict(df[['popularity', 'winrate', 'banrate',
                            'tsr', 'ts_buff', 'ts_nerf', 'vpop', 'vbr', 'vwr']].as_matrix())
prob_xgb_o = xgbc.predict_proba(df[['popularity', 'winrate', 'banrate', 'tsr', 'ts_buff', 'ts_nerf', 'vpop','vbr', 'vwr']].as_matrix())
dfp = pd.DataFrame(p_xgb_o, columns=['changes'])

dfn = pd.DataFrame(df.iloc[dfp[dfp['changes'] == 'nerf'].index]['name'])
# nerfs_html = dfn.transpose().to_html(index=False)

dfb = pd.DataFrame(df.iloc[dfp[dfp['changes'] == 'buff'].index]['name'])
# buffs_html = dfb.transpose().to_html(index=False)




In [36]:
# prob_xgb_o[df.iloc[dfp[dfp['changes'] == 'buff'].index].index]
# df.iloc[dfp[dfp['changes'] == 'nerf'].index]

# prob_xgb_o
# df_current = pd.concat([df['name'], prob_xgb_o])



,0,1,2,3,4,5,6,7,8,9,...,123,124,125,126,127,128,129,130,131,132
name,aatrox,ahri,akali,alistar,amumu,anivia,annie,ashe,aurelionsol,azir,...,warwick,xerath,xinzhao,yasuo,yorick,zac,zed,ziggs,zilean,zyra
buff,0.236957,0.0867475,0.140585,0.151372,0.304076,0.102903,0.103945,0.143602,0.170288,0.415141,...,0.576067,0.126072,0.196879,0.156454,0.260502,0.18715,0.071082,0.107339,0.103337,0.0949316


In [87]:
# df_nerf_probs['dates'] = pd.to_datetime(df_nerf_probs['dates'],unit='s').dt.date
# df_nerf_probs.head()

ValueError: non convertible value 2016-06-25with the unit 's'

In [96]:

# Format probabilities for predicted nerfs
df_nerf_probs = pd.read_csv('lolcurrent_nerf_probs.csv')
champs_nerf = list(dfn['name'].values)
champs_nerf.append('dates')
# df_nerf_probs = df_nerf_probs[champs_nerf]
df_nerf_probs['dates'] = pd.to_datetime(df_nerf_probs['dates'],unit='s').dt.date
# df_nerf_probs.to_csv('graph_nerfs_all.csv', index=False, na_rep='NaN')



# Format probabilities for predicted buffs
df_buff_probs = pd.read_csv('lolcurrent_buff_probs.csv')
champs_buff = list(dfb['name'].values)
champs_buff.append('dates')
# df_buff_probs = df_buff_probs[champs_buff]
df_buff_probs['dates'] = pd.to_datetime(df_buff_probs['dates'],unit='s').dt.date
# df_buff_probs.to_csv('graph_buffs_all.csv', index=False, na_rep='NaN')


# plt.figure(figsize=(20,10))
           
# for champ in df.iloc[dfp[dfp['changes']=='nerf'].index]['name']:
# #     champ = df.iloc[dfp[dfp['changes']=='nerf'].index]['name'].iloc[0]
#     data = df_probs[df_probs['name']==champ]['nerf']
#     if len(data) > 15:
#         ax = sns.tsplot(data=data, color = 'red')
        
# plt.legend()


,dates,lulu,skarner,varus,twitch,blitzcrank,annie,katarina,veigar,udyr,...,darius,taliyah,kogmaw,trundle,tristana,shen,kennen,missfortune,ahri,soraka
0,2016-06-25,0.085693,0.207152,0.312601,0.085677,0.783719,0.110994,0.201665,0.156975,0.296434,...,0.096875,0.170004,0.158110,0.941265,0.340586,0.128797,0.128596,0.156475,0.127383,0.034903
1,2016-07-05,0.045064,0.169600,0.372725,0.070176,0.873710,0.106621,0.180591,0.247621,0.489874,...,0.081873,0.073545,0.173085,0.930249,0.196096,0.236397,0.243280,0.173989,0.101121,0.039593
2,2016-07-13,0.193643,0.152703,0.318184,0.066010,0.146062,0.920253,0.235485,0.301660,0.428864,...,0.077573,0.065888,0.181564,0.335608,0.258240,0.209073,0.159978,0.228605,0.112351,0.063484
3,2016-07-20,0.262255,0.144049,0.285787,0.072398,0.186923,0.144403,0.172629,0.284802,0.317653,...,0.118604,0.040074,0.191172,0.302272,0.224522,0.041090,0.137863,0.196822,0.106508,0.041606
4,2016-07-30,0.338039,0.184986,0.372725,0.077141,0.158728,0.181503,0.146645,0.306366,0.307058,...,0.097830,0.045012,0.142916,0.402761,0.135816,0.040953,0.114512,0.239130,0.109142,0.045012
5,2016-08-08,0.944058,0.134613,0.401218,0.064418,0.122761,0.097974,0.125209,0.193880,0.261505,...,0.094244,0.081770,0.094060,0.423504,0.099352,0.166183,0.162504,0.169827,0.081060,0.051343
6,2016-08-18,0.855307,0.149096,0.404628,0.065205,0.137741,0.104248,0.217060,0.280694,0.339201,...,0.067799,0.090844,0.106927,0.310401,0.125632,0.161841,0.113880,0.221984,0.096816,0.048422
7,2016-08-31,0.445230,0.115562,0.408594,0.050673,0.122761,0.099688,0.170571,0.859760,0.373944,...,0.099883,0.090278,0.092318,0.919362,0.160512,0.156875,0.879648,0.211797,0.099546,0.047398
8,2016-09-11,0.852706,0.105035,0.292643,0.065070,0.150711,0.102302,0.161722,0.172819,0.443254,...,0.053223,0.626155,0.063178,0.348124,0.097002,0.069508,0.156572,0.837192,0.118403,0.041428
9,2016-09-20,0.295205,0.153586,0.327041,0.028555,0.075804,0.099399,0.097891,0.194410,0.950602,...,0.100364,0.309707,0.072563,0.370361,0.135620,0.176694,0.203393,0.176442,0.102163,0.011565


In [36]:
df_nerfs_all = pd.read_csv('graph_nerfs_all.csv')
df_buffs_all = pd.read_csv('graph_buffs_all.csv')

champs_nerf = list(dfn['name'].values)
champs_nerf.append('dates')
champs_buff = list(dfb['name'].values)
champs_buff.append('dates')

df_nerfs_index = df_nerfs_all[champs_nerf][-10:]
df_buffs_index = df_buffs_all[champs_buff][-10:]

df_buffs_index.to_csv('buffs_index.csv', index=False)
df_nerfs_index.to_csv('nerfs_index.csv', index=False)

In [88]:
df_nerf_probs = pd.read_csv('lolcurrent_nerf_probs.csv')

df_nerf_probs.iloc[-11:-1]

,dates,lulu,skarner,varus,twitch,blitzcrank,annie,katarina,veigar,udyr,...,darius,taliyah,kogmaw,trundle,tristana,shen,kennen,missfortune,ahri,soraka
20,1.483759e+09,0.109332,0.092036,0.081059,0.252759,0.158803,0.052595,0.584275,0.073459,0.093167,...,0.268252,0.089352,0.047427,0.069897,0.055723,0.019907,0.913194,0.100124,0.201233,0.174726
21,1.483822e+09,0.105598,0.148755,0.085949,0.251493,0.185876,0.082821,0.489564,0.067561,0.120705,...,0.316350,0.102762,0.123875,0.056968,0.055739,0.033924,0.912469,0.183819,0.207252,0.164910
22,1.483898e+09,0.107688,0.083158,0.073542,0.229788,0.316393,0.062640,0.383028,0.063079,0.148847,...,0.477102,0.124389,0.094446,0.040054,0.083571,0.024402,0.905582,0.117106,0.212491,0.106804
23,1.483998e+09,0.087095,0.065788,0.086602,0.164287,0.241707,0.081006,0.669896,0.067957,0.099838,...,0.390979,0.096206,0.180388,0.040501,0.075933,0.050506,0.911350,0.117773,0.168607,0.106538
24,1.484078e+09,0.093194,0.140669,0.063135,0.175897,0.178442,0.074718,0.584190,0.054345,0.120919,...,0.480024,0.068203,0.112870,0.055357,0.053569,0.047949,0.913375,0.101234,0.158308,0.161406
25,1.484158e+09,0.123685,0.098032,0.135719,0.237933,0.186400,0.066369,0.649049,0.087646,0.065600,...,0.502039,0.097553,0.096725,0.078786,0.041830,0.067864,0.915480,0.124836,0.207179,0.156133
26,1.484256e+09,0.097253,0.082377,0.179568,0.196412,0.228076,0.071920,0.621172,0.060260,0.158198,...,0.461962,0.089814,0.135797,0.038990,0.039771,0.056845,0.071556,0.166121,0.281356,0.106231
27,1.484398e+09,0.087603,0.058646,0.134785,0.170622,0.166513,0.099329,0.571818,0.085184,0.070491,...,0.468620,0.093386,0.104275,0.056174,0.079351,0.065068,0.053689,0.170793,0.323753,0.107169
28,1.484771e+09,0.089021,0.154966,0.122565,0.196865,0.204679,0.075994,0.523678,0.074298,0.177773,...,0.370843,0.108248,0.072676,0.044361,0.047395,0.076180,0.059683,0.153128,0.245815,0.106137
29,1.484875e+09,0.115122,0.101339,0.214969,0.165194,0.174481,0.079946,0.665471,0.071282,0.074536,...,0.378132,0.200880,0.134964,0.053162,0.042545,0.045515,0.091260,0.146898,0.242388,0.147819


In [77]:
# Add latest data

df_nerf_probs = pd.read_csv('lolcurrent_nerf_probs.csv')
df_buff_probs = pd.read_csv('lolcurrent_buff_probs.csv')

df_probs = pd.DataFrame(prob_xgb_o, columns=['buff', 'nerf', 'nochange'])
df_latest = pd.concat([df['name'], df_probs], axis=1)

df_buff_latest = df_latest[['name', 'buff']].transpose()
df_buff_latest.columns = df_buff_latest.iloc[0]
df_buff_latest.drop('name', inplace=True)
df_buff_latest.head()
df_buff_probs = pd.merge(df_buff_probs, df_buff_latest, on=None, how='outer')

df_nerf_latest = df_latest[['name', 'nerf']].transpose()
df_nerf_latest.columns = df_nerf_latest.iloc[0]
df_nerf_latest.drop('name', inplace=True)
df_nerf_latest.head()
df_nerf_probs = pd.merge(df_nerf_probs, df_nerf_latest, on=None, how='outer')


,dates,lulu,skarner,varus,twitch,blitzcrank,annie,katarina,veigar,udyr,...,darius,taliyah,kogmaw,trundle,tristana,shen,kennen,missfortune,ahri,soraka
0,1.466892e+09,0.869692,0.091341,0.097682,0.146590,0.125907,0.070553,0.072113,0.096337,0.066578,...,0.391201,0.269925,0.209274,0.026292,0.055113,0.225913,0.069076,0.091482,0.192824,0.291961
1,1.467760e+09,0.920610,0.111814,0.049251,0.196360,0.084512,0.093265,0.090524,0.058210,0.053064,...,0.444473,0.613033,0.146086,0.036696,0.070195,0.352802,0.089859,0.068270,0.153002,0.651765
2,1.468397e+09,0.156610,0.091184,0.130750,0.199197,0.137913,0.016794,0.096117,0.060149,0.055699,...,0.413885,0.900751,0.109009,0.073379,0.046295,0.317515,0.106148,0.072672,0.184159,0.367894
3,1.469041e+09,0.114573,0.122854,0.070029,0.221874,0.101292,0.048642,0.104549,0.054308,0.120702,...,0.289477,0.914164,0.063631,0.078036,0.100672,0.922714,0.138728,0.064006,0.181019,0.342781
4,1.469840e+09,0.110879,0.145899,0.049251,0.294619,0.178688,0.076875,0.115974,0.103880,0.062182,...,0.416574,0.895564,0.456496,0.096347,0.057994,0.922194,0.310743,0.126477,0.180700,0.553323
5,1.470614e+09,0.019931,0.106840,0.039476,0.300227,0.105273,0.121971,0.191764,0.106819,0.085801,...,0.331018,0.217340,0.473048,0.062727,0.126468,0.160731,0.193175,0.064672,0.202178,0.469540
6,1.471493e+09,0.048232,0.119949,0.090650,0.202201,0.130871,0.082490,0.168387,0.071722,0.056869,...,0.287453,0.589643,0.334678,0.089412,0.098168,0.166720,0.183737,0.079034,0.145428,0.499722
7,1.472663e+09,0.056392,0.332844,0.081737,0.592026,0.105273,0.164736,0.136052,0.055312,0.045839,...,0.358887,0.521871,0.537662,0.028515,0.114139,0.322123,0.062112,0.115046,0.179042,0.644692
8,1.473595e+09,0.057512,0.115750,0.069339,0.247490,0.159307,0.114634,0.077203,0.049636,0.052010,...,0.413584,0.264699,0.707475,0.053656,0.145628,0.866095,0.255272,0.059689,0.292931,0.641263
9,1.474405e+09,0.080156,0.172074,0.080620,0.865135,0.294810,0.099860,0.081071,0.039181,0.020336,...,0.320410,0.088693,0.266892,0.051082,0.093591,0.127956,0.320327,0.150115,0.200080,0.956111


In [71]:
# df_buff_probs.to_csv('lolcurrent_buff_probs.csv', index=False)
# df_nerf_probs.to_csv('lolcurrent_nerf_probs.csv', index=False)

In [53]:
df_buff_latest.head()

,0,1,2,3,4,5,6,7,8,9,...,123,124,125,126,127,128,129,130,131,132
name,aatrox,ahri,akali,alistar,amumu,anivia,annie,ashe,aurelionsol,azir,...,warwick,xerath,xinzhao,yasuo,yorick,zac,zed,ziggs,zilean,zyra
buff,0.236957,0.0867475,0.140585,0.151372,0.304076,0.102903,0.103945,0.143602,0.170288,0.415141,...,0.576067,0.126072,0.196879,0.156454,0.260502,0.18715,0.071082,0.107339,0.103337,0.0949316


In [33]:
df_probs = pd.read_csv('lolcurrent_probs.csv')
group = df_probs.groupby('name')

data = pd.DataFrame(df_probs['dates'].unique(), columns=['dates'])

for champ in list(group.groups.keys()):

    df_temp = group.get_group(champ)
    df_temp = df_temp[['dates', 'nerf']]
    df_temp.columns = ['dates', champ]

    data = pd.merge(data, df_temp, on='dates', how='outer')
data.to_csv('lolcurrent_nerf_probs.csv', index=False)


data = pd.DataFrame(df_probs['dates'].unique(), columns=['dates'])
for champ in list(group.groups.keys()):

    df_temp = group.get_group(champ)
    df_temp = df_temp[['dates', 'nerf']]
    df_temp.columns = ['dates', champ]

    data = pd.merge(data, df_temp, on='dates', how='outer')
data.to_csv('lolcurrent_buff_probs.csv', index=False)

,dates,singed,amumu,nautilus,diana,olaf,yorick,soraka,morgana,nunu,...,khazix,gragas,rengar,zyra,ekko,garen,teemo,poppy,evelynn,camille
0,1.466892e+09,0.071609,0.089024,0.056511,0.071921,0.066975,0.032173,0.291961,0.161948,0.108939,...,0.344901,0.165475,0.203174,0.159042,0.170283,0.033403,0.098187,0.066937,0.195211,NaN
1,1.467760e+09,0.152875,0.088062,0.072624,0.086101,0.203432,0.053871,0.651765,0.150986,0.090218,...,0.561865,0.373005,0.302725,0.078964,0.219171,0.088358,0.061591,0.068193,0.298716,NaN
2,1.468397e+09,0.074368,0.186705,0.071366,0.056007,0.137072,0.076856,0.367894,0.124214,0.061091,...,0.449456,0.386413,0.217538,0.073293,0.131607,0.050349,0.075752,0.024344,0.135717,NaN
3,1.469041e+09,0.157401,0.103515,0.109577,0.100099,0.253025,0.024411,0.342781,0.267285,0.060193,...,0.421260,0.582264,0.278898,0.021954,0.082170,0.058897,0.087937,0.038710,0.143887,NaN
4,1.469840e+09,0.099194,0.086772,0.065426,0.068711,0.164493,0.043428,0.553323,0.171964,0.089160,...,0.398185,0.329889,0.300143,0.164846,0.367530,0.079985,0.092392,0.033638,0.464570,NaN
5,1.470614e+09,0.069834,0.100495,0.024919,0.059760,0.084918,0.053141,0.469540,0.236377,0.063631,...,0.446159,0.287410,0.256052,0.056791,0.371939,0.046730,0.077678,0.031163,0.335787,NaN
6,1.471493e+09,0.106528,0.142454,0.102550,0.066030,0.120939,0.025302,0.499722,0.289078,0.065675,...,0.471634,0.282069,0.240743,0.055143,0.353628,0.096170,0.071329,0.030869,0.213920,NaN
7,1.472663e+09,0.055718,0.089511,0.053210,0.023811,0.063216,0.105007,0.644692,0.108901,0.047803,...,0.433453,0.267886,0.289000,0.067006,0.438700,0.042562,0.079321,0.042924,0.147927,NaN
8,1.473595e+09,0.124849,0.071851,0.025971,0.038094,0.078058,0.158434,0.641263,0.074653,0.093020,...,0.313415,0.185321,0.198472,0.079801,0.886663,0.139417,0.061460,0.124385,0.223899,NaN
9,1.474405e+09,0.054714,0.153080,0.040905,0.049692,0.205469,0.195247,0.956111,0.184093,0.078039,...,0.367430,0.143679,0.190480,0.085896,0.394857,0.047687,0.082464,0.092501,0.244098,NaN


In [73]:
# data.to_csv('lolcurrent_buff_probs.csv', index=False)
# data.to_csv('lolcurrent_nerf_probs.csv', index=False)
# data.to_csv('lolcurrent_nochange_probs.csv', index=False)

In [8]:
# import pandas as pd

# champ_names = pd.read_csv('champs.csv')

# champ_names.to_csv('champcommas.csv', line_terminator="',\n'", index=False)

In [28]:
import pandas as pd

champ = 'camille'
nerfs = pd.read_csv('./flasklol/app/static/data/graph_nerfs_all.csv')
dfn = nerfs[['dates', champ]]
# df.to_csv('./flasklol/app/static/data/nerfb.csv',index=False)
dfn

,dates,camille
0,2016-06-25,NaN
1,2016-07-05,NaN
2,2016-07-13,NaN
3,2016-07-20,NaN
4,2016-07-30,NaN
5,2016-08-08,NaN
6,2016-08-18,NaN
7,2016-08-31,NaN
8,2016-09-11,NaN
9,2016-09-20,NaN


In [ ]:
sudo docker run --rm -d --name flask1 -p 80:80 josephlee021/flaskimage:version005

In [ ]:
sudo docker exec -i -t flask1 /bin/bash

In [ ]:
sudo docker stop flask1
sudo docker build -t joe/flaskimage:version005 .

In [ ]:
sudo ssh -i insight.pem ec2-user@ec2-52-10-107-25.us-west-2.compute.amazonaws.com

# blue trinket.gg